In [7]:
import pandas as pd
import numpy as np
data = pd.read_json('data/dataset_1.json')

data["label_bin"] = np.where(data["label"] > 1, 1, 0)
data["display_text"] = [d[1]['text'][d[1]['displayTextRangeStart']: d[1]['getDisplayTextRangeEnd']] for d in data[["text","displayTextRangeStart", "getDisplayTextRangeEnd"]].iterrows()]
print("max text length", len(data.iloc[np.argmax(data['text'].to_numpy())]['text']))
max_display_text_length = len(data.iloc[np.argmax(data['display_text'].to_numpy())]['display_text'])
print("max display text length", max_display_text_length)
data
X = data.display_text
y = data.label


max text length 286
max display text length 270


In [6]:
X

0       MOST FUN DAY EVER HAHAHAH but seriously love s...
1       Consider giving to these privacy-focused chari...
2       Very brave of your both to share your story. I...
3       No, but willing to try. Good practice for the ...
4       Someone I know is taken to jail very late at n...
                              ...                        
6520    Learn about the state of #SCADA and the #vulne...
6521    [#Bestof2019] In June, we were at @pariscyberw...
6522    Vulnerabilities in other #IoT devices connecte...
6523    Chinese state 'likely' linked to cyber spies t...
6524    #CyberAlerts | Multiple security advisories: D...
Name: display_text, Length: 6525, dtype: object

In [8]:
from sentence_transformers import SentenceTransformer, util
import torch
model = SentenceTransformer('stsb-bert-base')


Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/377 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
model.max_seq_length = max_display_text_length

embedding_list = model.encode(X, show_progress_bar=True)


In [11]:
embedding_list

array([[ 0.01031118,  0.47366434,  0.5285841 , ...,  0.3557919 ,
         0.3511162 ,  0.32985434],
       [ 0.94506556,  0.17291027,  0.68434525, ..., -0.03435059,
        -0.94774526,  0.64350206],
       [ 0.0271307 ,  0.2565855 ,  0.6282008 , ...,  0.00828963,
         0.1732656 , -0.80317223],
       ...,
       [ 0.285472  , -0.24517831,  0.22439268, ..., -0.338788  ,
        -1.4717871 ,  0.431167  ],
       [ 0.8057044 ,  0.4535507 , -0.87348175, ..., -0.39328516,
         0.11977603,  0.5895106 ],
       [ 0.11876404, -0.08858778,  1.7711028 , ...,  0.3796738 ,
        -0.9112845 ,  0.3616097 ]], dtype=float32)